# Trabajando con datos vectoriales

Este ejercitario utiliza las tareas comunes de procesar datos espaciales vectoriales que incluyen:

1. Crear datos geográficos desde texto
2. Visualizar información basicas
3. Realizar JOIN espacial y consultas

Para responder cada pregunta trate crear celdas independientes.

Es necesario importar los siguientes paquetes iniciales para trabajar.

In [11]:
import geopandas # leer y escribir datos espaciales
import matplotlib.pyplot as plt # visualizar informacion
import pandas # para leer y escribir archivos de texto

# Para desplegar paginas web
from IPython.display import IFrame
%matplotlib inline

# Leer datos 

En muchos casos de analisis limpio, el analista ya posee datos preparados para su posterior analisis espacial. Puede que la información esta disponible en formatos conocidos (Ej. shapefiles, raster `tiff`, geopackage `.gpkg` o geojson). sin embargo, es importante entender como crear datos geográficos desde archivos de texto.

A continuación, utilizaremos unos datos de ejemplo de www.infocasas.com.py, con precios y proyectos de departamentos in Asunción, PY. Esta información sirve como ejemplo de datos que son distribuidos sin ningun formato espacial explicito. También se provee los barrios de asunción en poligonos.

_Nota_: 
Si *no encuentra* el archivo `inmuebles.json` asegurese de descargar los archivos `asuncion_barrios.zip` y `inmuebles.zip` necesarios y extrer los mismos en la misma ubicación `datos/`.

## Trabajando con inmuebles en JSON

### 1. Abrir los datos JSON
Utilice el archivo `datos/inmuebles.json` y abralo con `pandas.read_json` para crear una planilla de calculo de datos en Pandas. 

Puede verificar la ayuda ejecutando con `pandas.read_json?` o en la página https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_json.html. Para leer la informacion en este mismo apartado se puede utilizar en una celda individual.

```python
IFrame("https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_json.html", width=800, height=600)
```


Almacena la informacion en la variable `apto_json` y verifica la informacion para responder las siguientes preguntas:

- ¿Cuántas columnas posee el archivo y describe las más releventas para el analisis espacial precios/ubicación?
- ¿Cuántos apartamentos hay en el archivo?
- ¿Cuáles son las columnas que permiten georeferenciar la información?
- ¿Que tipo de datos poseen las columnas de precio y coordenadas?
- A simple vista, ¿en qué sistema de referencia están las coordenadas?

Utiliza las celdas de abajo para justificar la respuesta con el código necesario.

_Nota_:   
Puede utilizar la función `from_epsg` del módulo de `fiona.crs` para justificar su respuesta en cuanto al sistema de referencias. 

```python
from fiona.crs import from_epsg
```

### 2. Crear geometrias con las coordenadas

Una vez identificadas las columnas con datos geograficos se debe crear una `GeoDataFrame` para trabajar con los datos geograficos y las coordenadas. 

En una variable `geometria` guardar el valor de convertir las coordenadas x e y a tipo puntos de `shapely`.

_Nota_:
Se puede utilizar la funcion utilizada en clase para convertir de cadenas a puntos `Point` utilizando el modulo `shapely.geometry`.

```python
from shapely.geometry import Point
```

### 3. Crear un GeoDataFrame con datos limpios datos

Con las geometrias en la variable `geometria` se puede conbinar en una sola variable `inmuebles` la información de análisis por medio del objeto `geopandas.GeoDataFrame` pasando en primer lugar el archivo original y segundo las geometrias, pase los valores en orden:
Ej.

```python
inmuebles = geopandas.GeoDataFrame(apto_json, geometry=geometria)
```

Puede establecer el sistema de coordenadas con el parametro `crs` como tercer parametro o una vez definida la variable `inmuebles` definir la misma.

Para completar este paso se debe:
- a. Crear un GeoDataFrame con las coordenadas convertidas. Verificar la información con el método `.head()`
- b. Haber definido el sistema de coordenadas apropiado para el DataFrame
- c. Guardar el resultado en un archivo shapefile el nombre `"inmuebles.shp"` en la carpeta `datos/` 

_Nota_: Utilice la funcion `to_file` y el parametro `driver` si asi lo requiere con el valor "Esri Shapefile".

### 4. Limpiar datos

Los datos en muchos casos pueden no estar completamente limpios a simple vista. Luego de examinar la información acerca del precio se puede apreciar que los datos se encuentran en cierto formato. 

Ej.

```python
inmuebles["precio"].head(3)
```
El precio tiene un tipo no numérico como cadena, Ej.:

| precio |
|---:|
|U\$S 45.000|
|Gs. 395.000.000|
|Consultar|
|U\$S 125.000|


Pero se debe convertir a un formato numerico, Ej.:

45000

Lo que se requiere:
- Convertir la cadena a número utilizando los métodos de cadena como `.replace()` sobre la serie de la columna `precio`. Prestar atención al valor `"Consultar"` que debe reemplazar por `0`.
- Convertir la cadena a número `float` utilizando el método `.astype()` sobre la serie de la columna `precio`. 
- Crear la columna `precio_final` con los valores limpiados y convertidos. Verificar la informacion con el método `.head()`

_Nota_: 
Todos los métodos de cadena se pueden utilizar directamente sobre una columna de la siguiente manera. Ej.

```python
inmuebles["precio"].str.replace('AAA', 'BBB')
```



### 5. Analizar información

Ahora que tiene los precios finales se puede resumir los datos en base a diferentes monedas. 

Se puede dibujar un histograma con la función `hist` y un filtro de como sigue:

#### Histograma
Crear un histograma por medio de filtros utilizando la moneda del precio. Ej.

```python
filtro = inmuebles["precio"].str.startswith("Gs.")
inmuebles[filtro]["precio_final"].hist(bins=200)
```

Se requiere lo siguiente (crear una celda independiente para las graficas y resultados):
- Cree un filtro en la variable `es_dolar` por medio de la expresion `startswith` sobre la columna `precio`. 
- Cree un filtro en la variable `es_pyg` por medio de la expresion `startswith` sobre la columna `precio`. 
- Dibuje un histograma para la moneda U$S y para la Gs. en base al `precio_final` y verifique la información. 
- ¿Cuál es el departamento con mayor precio en USD dolares y cuanto es el precio final?
- ¿Cuál es el departamento con mayor precio en Gs. Guaraníes y cuanto es el precio final?

_Nota_: 
Puede verificar el valor maximo con el método `describe()` o por medio de un filtro con el método `idxmax()`.

Resumir los datos en dolares
```python
inmuebles[es_dolar]["precio_final"].describe()
```

Ver el mayor precio en moneda Gs.
```python
maximo = inmuebles[es_pyg]["precio_final"].idxmax()
inmuebles.iloc[maximo]
```

### Ver foto del departamento
Verificar la información de la foto ubicado en la columna `img` Ej.

```python
IFrame(src="imagen.png", width=400, height=400)
```

Para ayudar en este ejercicio cree dos variables `maximo_usd` y `maximo_pyg` con los valores de los indices de los departamentos con mayor precio en ambas monedas.

Se requiere lo siguiente (crear una celda independiente para cada pregunta)
- Visualice la imagen del departamento de mayor precio en guaraníes utilizando la funcion `IFrame`.
- Visualice la imagen del departamento de mayor precio en dolares utilizando la funcion `IFrame`.

_Nota_: 
Puede acceder directamente al registro filtrando por la columna `id` o el `precio` si conoce el valor del mismo.

Ej.
```python
# Filtro por ID
inmuebles[inmuebles.id == 186123414]
# Seleccion por precio
inmuebles[inmuebles.precio == "U$S 129.200"]
```
